<a href="https://colab.research.google.com/github/martinkenas/Road-Segmentation-for-type-wetness-and-roughness/blob/main/main_vid_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Galutinės sistemos kodas.
# Reikalingų bibliotekų įdiegimas ir importavimas
!pip install -q ultralytics opencv-python
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import time
from google.colab import files
from IPython.display import display
import ipywidgets as widgets

def upload_files():
    """Įkelti modelių ir vaizdo įrašo failus iš kompiuterio į Colab"""
    print("=" * 70)
    print("Kelio pavirsiaus aptikimas, tipo, dregnumo ir siurkstumo klasifikavimas")
    print("=" * 70)

    # Įkelti kelio aptikimo modelio failą
    print("\n1 zingsnis: Prasome ikelti YOLOv8-seg kelio aptikimo modelio faila (.pt)")
    detection_model_files = files.upload()
    detection_model_path = list(detection_model_files.keys())[0]

    # Įkelti kelio tipo klasifikavimo modelio failą
    print("\n2 zingsnis: Prasome ikelti YOLOv8-seg kelio tipo klasifikavimo modelio faila (.pt)")
    type_model_files = files.upload()
    type_model_path = list(type_model_files.keys())[0]

    # Įkelti kelio drėgnumo klasifikavimo modelio failą
    print("\n3 zingsnis: Prasome ikelti YOLOv8-seg kelio dregnumo klasifikavimo modelio faila (.pt)")
    wetness_model_files = files.upload()
    wetness_model_path = list(wetness_model_files.keys())[0]

    # Įkelti kelio šiurkštumo klasifikavimo modelio failą
    print("\n4 zingsnis: Prasome ikelti YOLOv8-seg kelio siurkstumo klasifikavimo modelio faila (.pt)")
    roughness_model_files = files.upload()
    roughness_model_path = list(roughness_model_files.keys())[0]

    # Įkelti vaizdo įrašo failą
    print("\n5 zingsnis: Prasome ikelti vaizdo iraso faila")
    video_files = files.upload()
    video_path = list(video_files.keys())[0]

    return detection_model_path, type_model_path, wetness_model_path, roughness_model_path, video_path

def filter_connected_components(mask, min_size_ratio=0.01):
    """
    Filtruoja mažus, neprijungtus komponentus iš kaukės.
    Palieka tik didžiausią prijungtą komponentą ir kitus, kurie viršija
    minimalaus dydžio slenkstį (išreikštą kaip bendrojo vaizdo dydžio santykis).
    """
    # Rasti visus prijungtus komponentus
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)

    if num_labels <= 1:  # Komponentų nerasta
        return mask

    # Apskaičiuoti minimalaus ploto slenkstį (proporcingai vaizdo dydžiui)
    min_size = mask.shape[0] * mask.shape[1] * min_size_ratio

    # Rasti didžiausią komponentą (išskyrus foną)
    largest_label = 1  # Pradedame nuo 1, nes 0 yra fonas
    largest_size = stats[1, cv2.CC_STAT_AREA]

    for i in range(2, num_labels):
        if stats[i, cv2.CC_STAT_AREA] > largest_size:
            largest_size = stats[i, cv2.CC_STAT_AREA]
            largest_label = i

    # Sukurti naują kaukę tik su reikšmingais komponentais
    filtered_mask = np.zeros_like(mask)

    # Visada įtraukti didžiausią komponentą
    filtered_mask[labels == largest_label] = 255

    # Įtraukti kitus komponentus, kurie viršija minimalų dydį
    for i in range(1, num_labels):
        if i != largest_label and stats[i, cv2.CC_STAT_AREA] >= min_size:
            filtered_mask[labels == i] = 255

    return filtered_mask

def process_video(detection_model_path, type_model_path, wetness_model_path, roughness_model_path, video_path, measure_perf=True):
    """
    Apdoroja vaizdo įrašą su kelio paviršiaus atpažinimu ir klasifikacija

    Parametrai:
    - detection_model_path: kelio aptikimo modelio failas
    - type_model_path: kelio tipo klasifikavimo modelio failas
    - wetness_model_path: kelio drėgnumo klasifikavimo modelio failas
    - roughness_model_path: kelio šiurkštumo klasifikavimo modelio failas
    - video_path: vaizdo įrašo failas
    - measure_perf: ar matuoti veikimo spartą (True/False)

    Grąžina:
    - vidutinį kadro apdorojimo laiką sekundėmis
    - output_path: apdoroto vaizdo įrašo kelias
    """
    # Sukurti išvesties failo pavadinimą
    output_path = "apdorotas_" + Path(video_path).name

    # Įkelti modelius - naudojant fiksuotas pasitikėjimo vertes
    print("Ikeliami modeliai...")
    detection_model = YOLO(detection_model_path)
    type_model = YOLO(type_model_path)
    wetness_model = YOLO(wetness_model_path)
    roughness_model = YOLO(roughness_model_path)

    # Apibrėžti tipo spalvas (BGR formatas OpenCV)
    type_colors = {
        0: (0, 255, 0),     # Žalia - "asfaltas"
        1: (42, 42, 165),   # Ruda - "žvyras"
        2: (128, 128, 128)  # Pilka - "grindinys"
    }

    # Apibrėžti drėgnumo spalvas (BGR formatas OpenCV)
    wetness_colors = {
        3: (255, 144, 30),  # Šviesiai mėlyna - "Sausas"
        4: (139, 0, 0)      # Tamsiai mėlyna - "Šlapias"
    }

    # Apibrėžti šiurkštumo spalvas (BGR formatas OpenCV) - ATNAUJINTI KLASIŲ ID
    roughness_colors = {
        5: (0, 255, 255),   # Geltona - "lygus"
        6: (0, 140, 255),   # Oranžinė - "šiurkštus"
        7: (0, 0, 255)      # Raudona - "labai šiurkštus"
    }

    type_names = {
        0: "asfaltas",
        1: "zvyras",
        2: "grindinys"
    }

    wetness_names = {
        3: "sausas",
        4: "slapias"
    }

    # ATNAUJINTI KLASIŲ ID šiurkštumui
    roughness_names = {
        5: "lygus",
        6: "siurkstus",
        7: "labai siurkstus"
    }

    # Atidaryti vaizdo įrašo failą
    print(f"Atidaromas vaizdo irasas: {video_path}")
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Klaida: Nepavyko atidaryti vaizdo iraso {video_path}")
        return 0, None

    # Gauti vaizdo įrašo savybes
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Nustatyti rodymo dimensijas


    # Nustatyti rodymo dimensijas
    panel_width = max(320, min(width, 640))
    panel_height = int(height * (panel_width / width))
    output_width = panel_width * 2
    output_height = panel_height * 2


    # Nustatyti vaizdo rašymo įrenginį
    out = cv2.VideoWriter(
        output_path,
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (output_width, output_height)
    )

    # Sukurti pažangos widget'ą
    progress_widget = widgets.FloatProgress(
        value=0,
        min=0,
        max=total_frames,
        description='Apdorojama:',
        bar_style='info'
    )
    status_widget = widgets.HTML(value="<h3>Apdorojamas vaizdo irasas...</h3>")
    display(status_widget)
    display(progress_widget)

    # Inicializuoti šriftą
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    font_thickness = 1
    font_color = (255, 255, 255)

    # FPS skaičiavimo kintamieji
    prev_time = time.time()
    current_fps = 0
    fps_alpha = 0.1  # FPS ekrano išlyginimui

    # Inicializuoti laikinojo glodinimo kintamuosius
    previous_road_masks = []  # Saugoti ankstesnių N kadrų kaukes
    temporal_window_size = 3  # Kadrų skaičius, į kurį reikia atsižvelgti atliekant laikiną glodinimą

    # Saugoti ankstesnes klasifikacijas laikinajam glodinimui
    prev_road_type = None
    prev_road_wetness = None
    prev_road_roughness = None
    type_smoothing_alpha = 0.7  # Svoris dabartiniam kadrui (1-alpha ankstesniam)
    wetness_smoothing_alpha = 0.6
    roughness_smoothing_alpha = 0.6

    # Našumo matavimo kintamieji
    processing_times = []

    try:
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Pradėti laiko matavimą kadro apdorojimui (neįskaitant vizualizacijos)
            if measure_perf:
                start_time = time.time()

            frame_count += 1
            progress_widget.value = frame_count

            # Periodiškai atnaujinti būsenos widget'ą
            if frame_count % 10 == 0:
                status_widget.value = f"<h3>Apdorojamas kadras {frame_count}/{total_frames}</h3>"

            # Pakeisti kadro dydį apdorojimui
            resized_frame = cv2.resize(frame, (panel_width, panel_height))

            # 1 žingsnis: Vykdyti kelio aptikimo išvadą su aukštesnėmis ribomis
            detection_results = detection_model.predict(
                resized_frame,
                conf=0.6,       # Aukštesnė pasitikėjimo riba
                iou=0.5,         # Aukštesnis IoU NMS
                verbose=False
            )
            detection_result = detection_results[0]

            # Išgauti kelio segmentavimo kaukę
            current_road_mask = np.zeros((panel_height, panel_width), dtype=np.uint8)

            if detection_result.masks is not None and len(detection_result.masks) > 0:
                for mask in detection_result.masks.data:
                    mask_array = mask.cpu().numpy()
                    mask_array = cv2.resize(mask_array, (panel_width, panel_height))
                    current_road_mask = np.logical_or(current_road_mask, mask_array > 0.5)
                current_road_mask = np.uint8(current_road_mask * 255)

                # Taikyti prijungtų komponentų filtravimą, kad būtų pašalinti maži, nesusiję segmentai
                current_road_mask = filter_connected_components(current_road_mask, min_size_ratio=0.02)

            # Taikyti laikinį glodinimą kelio kaukei
            previous_road_masks.append(current_road_mask)
            if len(previous_road_masks) > temporal_window_size:
                previous_road_masks.pop(0)  # Pašalinti seniausią kaukę

            # Apskaičiuoti laikinai išlygintą kaukę imant vidurkį
            if len(previous_road_masks) > 0:
                road_mask = np.zeros_like(current_road_mask, dtype=np.float32)
                for mask in previous_road_masks:
                    road_mask += mask.astype(np.float32)
                road_mask /= len(previous_road_masks)
                road_mask = (road_mask > 127).astype(np.uint8) * 255
            else:
                road_mask = current_road_mask

            # Sukurti maskuotą kadrą, kuriame yra tik kelias
            masked_frame = resized_frame.copy()
            for c in range(3):
                masked_frame[:,:,c] = cv2.bitwise_and(masked_frame[:,:,c], road_mask)

            # Inicializuoti vizualizacijos kadrus
            type_mask = np.zeros((panel_height, panel_width, 3), dtype=np.uint8)
            wetness_mask = np.zeros((panel_height, panel_width, 3), dtype=np.uint8)
            roughness_mask = np.zeros((panel_height, panel_width, 3), dtype=np.uint8)

            # Vykdyti klasifikavimo modelius tik jei kelias aptiktas
            road_type = None
            road_wetness = None
            road_roughness = None
            type_confidence = 0.0
            wetness_confidence = 0.0
            roughness_confidence = 0.0

            if np.sum(road_mask) > 0:
                # Vykdyti kelio tipo klasifikavimą
                type_results = type_model.predict(masked_frame, verbose=False)
                type_result = type_results[0]

                if type_result.boxes is not None and len(type_result.boxes) > 0:
                    classes = type_result.boxes.cls.cpu().numpy()
                    confidences = type_result.boxes.conf.cpu().numpy()

                    if len(classes) > 0:
                        best_idx = np.argmax(confidences)
                        current_type = int(classes[best_idx])
                        current_type_conf = confidences[best_idx]

                        # Taikyti laikinį glodinimą tipo klasifikacijai
                        if prev_road_type is not None:
                            # Pakeisti tipą tik jei pasitikėjimas yra didesnis nei ankstesnis arba virš slenksčio
                            if current_type_conf > 0.7 or current_type == prev_road_type:
                                road_type = current_type
                                type_confidence = current_type_conf
                            else:
                                road_type = prev_road_type
                                type_confidence = max(0.5, current_type_conf)  # Palaikyti tam tikrą pasitikėjimą
                        else:
                            road_type = current_type
                            type_confidence = current_type_conf

                        prev_road_type = road_type

                # Vykdyti kelio drėgnumo klasifikavimą
                wetness_results = wetness_model.predict(masked_frame, verbose=False)
                wetness_result = wetness_results[0]

                if wetness_result.boxes is not None and len(wetness_result.boxes) > 0:
                    classes = wetness_result.boxes.cls.cpu().numpy()
                    confidences = wetness_result.boxes.conf.cpu().numpy()

                    if len(classes) > 0:
                        best_idx = np.argmax(confidences)
                        # Žemėlapio klasės ID į naują diapazoną (3,4)
                        original_wetness = int(classes[best_idx])
                        current_wetness_conf = confidences[best_idx]

                        # Jei originalios klasės buvo 0,1, žemėlapyje jas 3,4
                        if original_wetness == 0:
                            current_wetness = 3
                        elif original_wetness == 1:
                            current_wetness = 4
                        else:
                            current_wetness = original_wetness  # Palikti kaip yra, jei jau tinkamame diapazone

                        # Taikyti laikinį glodinimą drėgnumo klasifikacijai
                        if prev_road_wetness is not None:
                            # Pakeisti drėgnumą tik jei pasitikėjimas yra didesnis nei ankstesnis arba virš slenksčio
                            if current_wetness_conf > 0.7 or current_wetness == prev_road_wetness:
                                road_wetness = current_wetness
                                wetness_confidence = current_wetness_conf
                            else:
                                road_wetness = prev_road_wetness
                                wetness_confidence = max(0.5, current_wetness_conf)
                        else:
                            road_wetness = current_wetness
                            wetness_confidence = current_wetness_conf

                        prev_road_wetness = road_wetness

                # Vykdyti kelio šiurkštumo klasifikavimą
                roughness_results = roughness_model.predict(masked_frame, verbose=False)
                roughness_result = roughness_results[0]

                if roughness_result.boxes is not None and len(roughness_result.boxes) > 0:
                    classes = roughness_result.boxes.cls.cpu().numpy()
                    confidences = roughness_result.boxes.conf.cpu().numpy()

                    if len(classes) > 0:
                        best_idx = np.argmax(confidences)
                        original_roughness = int(classes[best_idx])
                        current_roughness_conf = confidences[best_idx]

                        # Žemėlapio originalūs šiurkštumo klasės (0,1,2) į naują diapazoną (5,6,7)
                        if original_roughness == 0:
                            current_roughness = 5  # lygus -> 5
                        elif original_roughness == 1:
                            current_roughness = 6  # šiurkštus -> 6
                        elif original_roughness == 2:
                            current_roughness = 7  # labai šiurkštus -> 7
                        else:
                            current_roughness = original_roughness + 5  # Palikti sužemėlapyje, jei jau tinkamame diapazone

                        # Taikyti laikinį glodinimą šiurkštumo klasifikacijai
                        if prev_road_roughness is not None:
                            # Pakeisti šiurkštumą tik jei pasitikėjimas yra didesnis nei ankstesnis arba virš slenksčio
                            if current_roughness_conf > 0.7 or current_roughness == prev_road_roughness:
                                road_roughness = current_roughness
                                roughness_confidence = current_roughness_conf
                            else:
                                road_roughness = prev_road_roughness
                                roughness_confidence = max(0.5, current_roughness_conf)
                        else:
                            road_roughness = current_roughness
                            roughness_confidence = current_roughness_conf

                        prev_road_roughness = road_roughness

                # IŠLAIKYTI SANTYKIO TAISYKLES TARP TIPO IR ŠIURKŠTUMO
                if road_type is not None and road_roughness is not None:
                    # Taisyklė 1: Asfaltas (0) negali būti labai šiurkštus (7)
                    if road_type == 0 and road_roughness == 7:
                        road_roughness = 6

                    # Taisyklė 2: Žvyras (1) arba grindinys (2) negali būti lygus (5)
                    if (road_type == 1 or road_type == 2) and road_roughness == 5:
                        road_roughness = 6

            # Užbaigti laiko matavimą kadro apdorojimui (neįskaitant vizualizacijos)
            if measure_perf:
                end_processing_time = time.time()
                processing_time = end_processing_time - start_time
                processing_times.append(processing_time)

            # Sukurti vizualizaciją (šis laikas neįtraukiamas į našumo matavimą)

            # Taikyti tipo spalvų perdangą
            if road_type is not None and road_type in type_colors:
                type_color = np.array(type_colors[road_type])
                colored_type_mask = np.zeros_like(type_mask)
                for c in range(3):
                    colored_type_mask[:,:,c] = type_color[c]
                road_mask_3d = np.stack([road_mask, road_mask, road_mask], axis=2)
                type_mask = cv2.bitwise_and(colored_type_mask, road_mask_3d)

            # Taikyti drėgnumo spalvų perdangą
            if road_wetness is not None and road_wetness in wetness_colors:
                wetness_color = np.array(wetness_colors[road_wetness])
                colored_wetness_mask = np.zeros_like(wetness_mask)
                for c in range(3):
                    colored_wetness_mask[:,:,c] = wetness_color[c]
                road_mask_3d = np.stack([road_mask, road_mask, road_mask], axis=2)
                wetness_mask = cv2.bitwise_and(colored_wetness_mask, road_mask_3d)

            # Taikyti šiurkštumo spalvų perdangą
            if road_roughness is not None and road_roughness in roughness_colors:
                roughness_color = np.array(roughness_colors[road_roughness])
                colored_roughness_mask = np.zeros_like(roughness_mask)
                for c in range(3):
                    colored_roughness_mask[:,:,c] = roughness_color[c]
                road_mask_3d = np.stack([road_mask, road_mask, road_mask], axis=2)
                roughness_mask = cv2.bitwise_and(colored_roughness_mask, road_mask_3d)

            # Sukurti vizualinį originalaus kadro su aptiktu kelio kaukės kontūru perdangą derinimui
            road_overlay = resized_frame.copy()

            # Rasti kelio kaukės kontūrus kontūro vizualizacijai
            contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(road_overlay, contours, -1, (0, 255, 255), 2)  # Geltonas kontūras

            # Sukurti 2x2 tinklelio ekraną
            top_row = np.hstack((road_overlay, type_mask))  # Rodyti kelio kontūrą originalame kadre
            bottom_row = np.hstack((wetness_mask, roughness_mask))
            display_frame = np.vstack((top_row, bottom_row))

            # Apskaičiuoti FPS
            current_time = time.time()
            elapsed_time = current_time - prev_time
            if elapsed_time > 0:
                current_fps = current_fps * (1 - fps_alpha) + (1.0 / elapsed_time) * fps_alpha
            prev_time = current_time

            # Pridėti teksto etiketes
            cv2.putText(display_frame, "Originalas + Kelio Konturas", (panel_width//2 - 100, 30), font, font_scale, font_color, font_thickness)
            cv2.putText(display_frame, "Kelio Tipas", (panel_width + panel_width//2 - 50, 30), font, font_scale, font_color, font_thickness)
            cv2.putText(display_frame, "Kelio Dregnumas", (panel_width//2 - 60, panel_height + 30), font, font_scale, font_color, font_thickness)
            cv2.putText(display_frame, "Kelio Siurkstumas", (panel_width + panel_width//2 - 70, panel_height + 30), font, font_scale, font_color, font_thickness)

            # Pridėti FPS skaitiklį viršutiniame kairiajame originalaus vaizdo kampe
            cv2.putText(
                display_frame,
                f"FPS: {current_fps:.1f}",
                (10, 30),
                font, font_scale, (0, 255, 0), font_thickness
            )

            # Pridėti klasifikavimo rezultatus su pasitikėjimu
            if road_type is not None:
                cv2.putText(
                    display_frame,
                    f"{type_names.get(road_type, 'Nezinomas')} (Pasit.: {type_confidence:.2f})",
                    (panel_width + 10, panel_height - 20),
                    font, font_scale, font_color, font_thickness
                )

            if road_wetness is not None:
                cv2.putText(
                    display_frame,
                    f"{wetness_names.get(road_wetness, 'Nezinomas')} (Pasit.: {wetness_confidence:.2f})",
                    (10, panel_height + panel_height - 20),
                    font, font_scale, font_color, font_thickness
                )

            if road_roughness is not None:
                cv2.putText(
                    display_frame,
                    f"{roughness_names.get(road_roughness, 'Nezinomas')} (Pasit.: {roughness_confidence:.2f})",
                    (panel_width + 10, panel_height + panel_height - 20),
                    font, font_scale, font_color, font_thickness
                )

            # Įrašyti kadrą į išvesties vaizdo įrašą
            out.write(display_frame)

        # Atlaisvinti išteklius
        cap.release()
        out.release()

        # Atnaujinti būseną
        status_widget.value = f"<h3>Apdorojimas baigtas!</h3>"
        progress_widget.bar_style = 'success'
        progress_widget.value = total_frames

        print("=" * 70)
        print(f"Apdorojimas baigtas!")
        print(f"Rezultatas issaugotas: {output_path}")
        print("=" * 70)

        # Apskaičiuoti našumo statistiką
        if measure_perf and processing_times:
            avg_time = sum(processing_times) / len(processing_times)
            print(f"Vidutinis kadro apdorojimo laikas: {avg_time:.4f} sekundes")
            print(f"FPS (be vizualizacijos): {1.0/avg_time:.2f}")

            # Pateikti našumo ataskaitą
            perf_html = f"""
            <div style="background-color: #f0f0f0; padding: 15px; border-radius: 10px;">
              <h3>Nasumo ataskaita</h3>
              <p>Vidutinis kadro apdorojimo laikas: <b>{avg_time:.4f}</b> sekundes</p>
              <p>Vidutinis FPS (kadrai per sekunde): <b>{1.0/avg_time:.2f}</b></p>
              <p>Apdorota kadru: <b>{len(processing_times)}</b></p>
            </div>
            """
            display(widgets.HTML(value=perf_html))

            return avg_time, output_path
        else:
            return 0, output_path

        # Atsisiuntimo mygtukas
        download_button = widgets.Button(
            description='Atsisiusti apdorota vaizdo irasa',
            button_style='success',
            icon='download'
        )

        def on_button_click(b):
            files.download(output_path)

        download_button.on_click(on_button_click)
        display(download_button)

    except Exception as e:
        status_widget.value = f"<h3>Klaida: {str(e)}</h3>"
        progress_widget.bar_style = 'danger'
        print(f"Klaida apdorojant vaizdo irasa: {str(e)}")
        return 0, None

def main():
    # Įkelti failus
    detection_model_path, type_model_path, wetness_model_path, roughness_model_path, video_path = upload_files()

    # Apdoroti vaizdo įrašą ir išmatuoti našumą
    avg_processing_time, output_path = process_video(
        detection_model_path,
        type_model_path,
        wetness_model_path,
        roughness_model_path,
        video_path,
        measure_perf=True
    )

    if avg_processing_time > 0:
        print(f"Vidutinis kadro apdorojimo laikas: {avg_processing_time:.4f} sekundes")
        print(f"Vidutinis FPS: {1.0/avg_processing_time:.2f}")

    if output_path:
        print(f"Apdoroto vaizdo įrašo kelias: {output_path}")

        # Pasiūlyti atsisiųsti failą
        download_button = widgets.Button(
            description='Atsisiųsti apdorotą vaizdo įrašą',
            button_style='success',
            icon='download'
        )

        def on_download_click(b):
            files.download(output_path)

        download_button.on_click(on_download_click)
        display(download_button)
    else:
        print("Nepavyko apdoroti vaizdo įrašo.")

# Paleisti programą, kai paleidžiamas šis skriptas
if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultraly

Saving Galutinis_Pavirsiaus_Nustatymas.pt to Galutinis_Pavirsiaus_Nustatymas.pt

2 zingsnis: Prasome ikelti YOLOv8-seg kelio tipo klasifikavimo modelio faila (.pt)


Saving Galutinis_Tipo_Nustatymas.pt to Galutinis_Tipo_Nustatymas.pt

3 zingsnis: Prasome ikelti YOLOv8-seg kelio dregnumo klasifikavimo modelio faila (.pt)


Saving Galutinis_Bukles_Nustatymas.pt to Galutinis_Bukles_Nustatymas.pt

4 zingsnis: Prasome ikelti YOLOv8-seg kelio siurkstumo klasifikavimo modelio faila (.pt)


Saving Kelio_Siurkstumas_FINAL.pt to Kelio_Siurkstumas_FINAL.pt

5 zingsnis: Prasome ikelti vaizdo iraso faila


Saving VID_20250518_130509.mp4 to VID_20250518_130509.mp4
Ikeliami modeliai...
Atidaromas vaizdo irasas: VID_20250518_130509.mp4


HTML(value='<h3>Apdorojamas vaizdo irasas...</h3>')

FloatProgress(value=0.0, bar_style='info', description='Apdorojama:', max=2648.0)